In [261]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import librosa
import sklearn
import soundfile as sf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [262]:
DATADIR = "data"
CATEGORIES = ["Segmented_Laugh", "Segmented_NonLaugh"]

In [263]:
n_mfcc = 3


In [264]:
training_data = []
def create_training_data():
    count=0
    error_count = 0
    #Enter time in seconds
    t_window = 1
    t_frame = 0.025
    t_shift = 0.01
    n_mfccs = n_mfcc
    laugh_counter = 0
    nonlaugh_counter = 0
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) 
        class_num = CATEGORIES.index(category)
        print('New Folder')

        for aud in os.listdir(path):
            if aud == '.DS_Store':
                continue
            
            aud_array , sr = librosa.load(os.path.join(path,aud), sr=None)
            count+=1

            mfccs = []
            
            try:
                mfcc_temp = (librosa.feature.mfcc(aud_array, sr=sr,  n_mfcc=n_mfccs,  win_length = int(sr*t_frame), hop_length = int(sr*t_shift))) 
                mfcc_delta = librosa.feature.delta(mfcc_temp)
                
                mean_mfccs = np.mean(np.asarray(mfcc_temp),axis = 1)
                var_mfccs = np.var(np.asarray(mfcc_temp), axis = 1)
                std_deltamfccs = np.std(np.asarray(mfcc_delta), axis = 1)
                
                mfccs.append(mean_mfccs)
                mfccs.append(var_mfccs)
                mfccs.append(std_deltamfccs)
                mfccs = np.asarray(mfccs).reshape(n_mfccs*3,1)
                training_data.append([mfccs.reshape(-1,1), class_num])
                if category == 'Segmented_Laugh':
                    laugh_counter +=1
                else:
                    nonlaugh_counter += 1

            except ValueError:
                print("Oops!  That was no valid number.  Try again...")
        
    print(laugh_counter, nonlaugh_counter)

In [265]:

create_training_data()
data = np.array(training_data)

New Folder
New Folder
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
616 604


In [266]:
# mfcc array | class
random.shuffle(training_data)

In [267]:
data[0,0].shape

(9, 1)

In [268]:

X = []
Y = []
for features, label in training_data:
    X.append(features)
    Y.append(label)

    
X = np.array(X).reshape(-1,n_mfcc*3)
#-1 corresponds to how many features we have
Y = np.array(Y)
#X = X/250
#print(X.shape)
#print(Y)
X.shape

(1220, 9)

In [269]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size=0.1, 
                                                    random_state=4)

In [270]:
from sklearn.neighbors import KNeighborsClassifier



In [271]:
knn = KNeighborsClassifier()

In [272]:
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)

In [273]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[46 17]
 [22 37]]
              precision    recall  f1-score   support

           0       0.68      0.73      0.70        63
           1       0.69      0.63      0.65        59

    accuracy                           0.68       122
   macro avg       0.68      0.68      0.68       122
weighted avg       0.68      0.68      0.68       122



In [274]:
counter = 0
correct = 0
total = 0
chosenclass = 1
for y in y_test:
    if y == chosenclass:
        total += 1
        if y_pred[counter]== y:
            correct += 1
    counter += 1

In [275]:
correct

37

In [276]:
total

59